In [ ]:
!pip install transformers[all]

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 8.2 MB/s 


In [ ]:
!pip install transformers[sentencepiece]


In [ ]:
ch_text = 'hello 我叫萨拉，我住在伦敦。'

## Code


In [ ]:
!pip install python-magic

In [ ]:
!sudo apt-get install libmagic1

In [ ]:
import subprocess as sub

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
#import magic
#import shlex
import os
import re
import pickle
import fileinput
import sys

### Load model once and use it  

In [ ]:
zh2en_model = AutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-zh-en')
zh2en_tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-zh-en')

saved_model = pickle.dumps(zh2en_model)
saved_tokenizer = pickle.dumps(zh2en_tokenizer)

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/786k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/788k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

### Translate chinese text using saved model 

In [ ]:
def translate_text(ch_text):
  """
  pass your chinese text and it returns its english translation 
  """

  zh2en_model = pickle.loads(saved_model)
  zh2en_tokenizer = pickle.loads(saved_tokenizer)
  zh2en_translation = pipeline('translation_zh_to_en', 
                      model=zh2en_model, 
                      tokenizer=zh2en_tokenizer)
  return zh2en_translation(ch_text)

### get type of file to detect its comment character (NOT USED)

*will be better if it detects file type by its content not its name and extension*

In [ ]:
def get_type_of_file(data_path):
  """
  pass file path with its .format otherwise it faces some problems
  """
  try:
    #return magic.Magic().from_file(data_path)   
    #
    # p = sub.Popen(data_path , stdout=sub.PIPE, stderr=sub.PIPE)
    # output, errors = p.communicate()
    #
    # filename = data_path
    # cmd = shlex.split('file --mime-type {0}'.format(filename))
    # result = sub.check_output(cmd)
    # mime_type = result.split()[-1]
    # return mime_type
    file_name, file_extension = os.path.splitext(data_path)
    return file_extension
  except IOError:
    return 'No File Found'

In [ ]:
def get_index_of_chinese_text(file):


### Define a dictionary of file type and its comment character

In [ ]:
comment_format_start = {'.html':['<!--'],'.htm':['<!--'],'js':['//','/*'],'.java':['//','/*'],'.py':['#']}
comment_format_end = {'.html':'-->','.htm':'-->','js':'*/','java':'*/'}


### Searches for chinese characters and returns a modified translated same text 

In [ ]:
def replace_text(line):
  """
  pass line you want to change and the text before to replace new line 
  in new text
  """

  if re.search('[\u4e00-\u9fff]',line) != None:
    #line = line.replace('Hello', 'import')
    #sys.stdout.write(line)

    changes = line.replace(line, translate_text(line)[0]['translation_text'])
    return changes 

  else:
    return line 

### Main process of parsing text and creating new text file

In [ ]:
def process_text(input_path):
  try:
    f = open(input_path,'r')
    #get type of file based on its name (name.type)
    file_name, file_type = os.path.splitext(input_path)
    file_type = file_type[1:].lower()

    #flag to check if a multiple line comment is done or not
    comment_closed = True

    #new text file to be written
    replacement = []

    #flag to check if line is the first line of file
    first_line = True

    #try:

    #if the file type contains comment or not (like .md or .txt files)
    #if the file is comment included, checks all possible comment formats 
    if comment_format_start.get(file_type) != None:
      comment_format_s = comment_format_start[file_type]
      #print(comment_format_s)

      #read file line by line
      for line in f.readlines():
        #print(line)

        #checks if it's in the multiple line comment, 
        #if not, one line comment cases should be checked
        if comment_closed:
          
          # for file types containing different comment characters
          for i in comment_format_s:

            #if the comment started from first of line
            if line.strip().startswith(i):
              replacement.append("{}".format(i) + replace_text(line.split(i)[1]))
              #print('startswith{}'.format(i))
              #try:

              #searches for ending comment character, if the file type supports
              if comment_format_start.get(file_type) != None :
                comment_format_e = comment_format_end[file_type]
                #print('endswith{}'.format(comment_format_e))

                #if the line hasn't been ended with ending comment character, next lines should be considered as comment
                if not line.endswith(comment_format_e) and len(comment_format_s)>1:
                  if i == comment_format_s[1]:
                    comment_closed = False
              #except NameError:
              else:
                print('one line comment')

            #if comment started from middle of line
            if len(line.strip().split(i))>1 and line.strip().split(i)[0] != "" :
              replacement.append(line.strip().split(i)[0]+"{}".format(i)+replace_text(line.strip().split(i)[1]))
              #print('contains one line comment')


            #else:
          #if it's first line translate it by the way
          if first_line:
            replacement.append(replace_text(line))

          #append the line to new text if it's not repeated   
          if len(replacement)>0:
            if replacement[-1] != line[:line.index('\n')]:
              replacement.append(line)
              #print('no comment found')

        #We are in a multiple line comment, so it should translate it all till it sees ending comment character
        else:
          replacement.append(replace_text(line))
          #print('comment continues')

          if line.endswith(comment_format_e):
            comment_closed = True

        first_line = False
    #except NameError:

    #file doesn't contain comments and we should search for chinese characters whole th file
    else:
      #print('This file doesnt contain comment')
      for line in f.readlines():
        replacement.append(replace_text(line))
    f.close()


    #write new text in a translated file
    if len(replacement) != 0:
      fout = open("{}_translated.{}".format(file_name,file_type), "w")
      fout.writelines(replacement)
      fout.close()  

  except IOError:
    print('No File Found')



### Run this! 

In [13]:
process_text('main.js')